In [1]:
import os
import dask.array as da
import numpy as np

from pyralysis.io.antenna_config_io import AntennaConfigurationIo
from pyralysis.models.sky import PointSource
from pyralysis.models.sky import GaussianSource
from pyralysis.models.sky import CompositeSource
from pyralysis.simulation.core import Simulator

from astropy.time import Time
from astropy.coordinates import Angle
import astropy.units as u
from astropy.constants import c

In [2]:
notebook_path = os.path.abspath("")
existing_cfg_filename = "../antenna_configs/alma.cycle10.1.cfg"
existing_cfg_path = os.path.join(notebook_path, existing_cfg_filename)

print(existing_cfg_path)

/home/esteban/Escritorio/Esteban - USACH/2025.02/bda-interferometry/notebooks/../antenna_configs/alma.cycle10.1.cfg


In [3]:
da.random.seed(777)
np.random.seed(777)

io_instance = AntennaConfigurationIo(input_name=existing_cfg_path)
interferometer = io_instance.read()

freq = np.linspace(35, 50, 50) * u.GHz  # ALMA Band 1
ref_freq = np.median(freq)

nodim_freq = freq.to(u.Hz).value
print("Number of frequencies:", len(nodim_freq))
print("Reference frequency:", ref_freq)

diameter = interferometer.antenna_array.diameters.compute()

fov = (c.value / nodim_freq[0]) / diameter[0]
print("FOV (radians): ", fov)
date_string = "2002-05-10"

interferometer.configure_observation(
    frequencies=freq,
    reference_time=Time(date_string, format='iso'),
    observation_time='4h',
    declination=Angle("-45d00m00s"),
    frequency_step_hz=None,
    integration_time=180 * u.s,
)

n_sources = np.random.randint(1, high=20, size=1, dtype=int)[0]
print("Number of point sources: ", n_sources)

s_0 = 1.0 * u.mJy  # flux density of all sources
l_0 = np.random.uniform(low=-fov, high=fov, size=n_sources)  # l in radians for all sources
m_0 = np.random.uniform(low=-fov, high=fov, size=n_sources)  # m in radians

l_0_gaussian = np.random.uniform(low=-fov, high=fov, size=1)  # l in radians for all sources
m_0_gaussian = np.random.uniform(low=-fov, high=fov, size=1)  # m in radians

sources = []
for i in range(n_sources):
    sources.append(
        PointSource(
            reference_intensity=s_0,
            spectral_index=3.0,
            reference_frequency=ref_freq,
            direction_cosines=(l_0[i], m_0[i])
        )
    )

gaussian_source = GaussianSource(
    reference_intensity=10 * u.Jy,
    direction_cosines=(0, 0),
    minor_radius=Angle(20 * u.arcsec),
    major_radius=Angle(30 * u.arcsec),
    theta_angle=Angle(60 * u.deg)
)

sources.append(gaussian_source)

print("Total number of sources: ", len(sources))

composite_source = CompositeSource(sources=sources)

sim = Simulator(interferometer=interferometer, sources=composite_source)

Number of frequencies: 50
Reference frequency: 42.5 GHz
FOV (radians):  0.0007137915666666667
Number of point sources:  8
Total number of sources:  9


In [ ]:
simulated_dataset = sim.simulate()

simulated_dataset